In [1]:
import pandas as pd
import numpy as np
import glob
import random
import pickle
import copy

In [2]:
gamma = 0.99
n_memory = 1000000
explor = False
fit_count = 0
best_advanced_step=0
bestofbest_step = 0
exploration_rate =0

In [3]:
# temp = r'input_data\*csv'
# csv_dir = glob.glob(temp)
# for i in range(len(csv_dir)):
#     print(i, csv_dir[i])

In [4]:
# df_list = []
# for i in range(len(csv_dir)):
#     data = pd.read_csv(csv_dir[i], header = None)
#     tmp = data.iloc[:,0].unique()
#     for j in range(len(tmp)):
#         temp = data[data[0]==tmp[j]].set_index(1)
#         temp.columns = ['prod_name', 'z_axis', 'CentSFR 0.7']
#         df_list.append(temp)

In [5]:
# with open('df_list.pickle', 'wb') as f:
#     pickle.dump(df_list, f)
with open('df_list.pickle', 'rb') as f:
    df_list = pickle.load(f)

In [6]:
df_list[50000]

,prod_name,z_axis,CentSFR 0.7
1,,,
0,20223M133784,-10.350,0.105402
1,20223M133784,-10.375,0.112794
2,20223M133784,-10.400,0.238004
3,20223M133784,-10.410,0.374730
4,20223M133784,-10.420,0.517354
5,20223M133784,-10.430,0.654284
6,20223M133784,-10.440,0.766055
7,20223M133784,-10.450,0.833290
8,20223M133784,-10.460,0.845576


In [ ]:
# 각각 한 개가 simulation environment 임.
len(df_list)

52962

In [8]:
# 70퍼센트만 학습에 사용
train_test_ratio = 0.7
idx = np.arange(0,len(df_list))
random.shuffle(idx)
train_df_list=[]
test_df_list=[]
for i in range(int(len(df_list)*train_test_ratio)):
    train_df_list.append(df_list[idx[i]])
for i in range(int(len(df_list)*train_test_ratio),len(df_list)):
    test_df_list.append(df_list[idx[i]])

In [9]:
# current state 에 따라 어떤 행동을 할 지 결정 (exploration_rate 전역변수)
def decide(model, cur_state):
    global explor
    if random.random()>exploration_rate:
        tmp = model.predict_on_batch(cur_state)
        action = tmp.argmax()+1
        explor = False
    else:
        action = random.randint(1,5)
        explor = True
    return action

In [10]:
# exploration 을 하지 않는 경우 (exploration_rate 이 음수인 경우와 동일.)
def best_decide(model, cur_state):

    tmp = model.predict_on_batch(cur_state)
    action = tmp.argmax()+1
    explor = False

    return action

In [11]:
# df_list 에서 환경 하나를 랜덤으로 추출해 세팅함.
# step : 실제 best step.
# value : 실제 best step에서의 해상도
#  tmp : 환경
def random_load(df_list):
    i=1
    while i!=0:
        tmp = df_list[random.randint(0,len(df_list)-1)]
        i = tmp.index[0]
    
#     시작지점이 최고해상도인 경우가 있음.
#  이 경우 실제로 z axis 를 당겨서 다시 측정함.
    if tmp['CentSFR 0.7'].argmax() == 0:
        step = tmp.iloc[2:,2].argmax()+2
    else:
        step = tmp['CentSFR 0.7'].argmax()
    
    value = tmp.iloc[step,:]['CentSFR 0.7']
    return [tmp, step, value]

In [12]:
# df_list 에서 k index를 주어 환경을 특정해서 뽑는 경우
def select_load(df_list,k):
    i=1
    while i!=0:
        tmp = df_list[k]
        i = tmp.index[0]
    if tmp['CentSFR 0.7'].argmax() == 0:
        step = tmp.iloc[2:,2].argmax()+2
    else:
        step = tmp['CentSFR 0.7'].argmax()
    
    value = tmp.iloc[step,:]['CentSFR 0.7']
    return [tmp, step, value]

In [13]:
# 처음 시작할 때 cur_env 생성
# cur_env : [현재 step, 현재 해상도, 실제 best step]
def init_env(data):
    return [0, data[0].iloc[0,2], data[1]]

In [ ]:
# 액션을 취했을 때 환경을 변화시키고, reward 를 관찰해 memory_D에 적재.

# cur_state: [[현재 z 거리, 바로 전 z 거리, 현재 SFR, 바로 전 SFR, 기울기]]
def update_env(data,cur_env,action,memory_D, cur_state):
    global terminal, fit_count, succeeded, failed
    tmp=[]
    tmp.append(cur_state.copy())
    tmp.append(action)
    
#     액션 진행
    cur_env[0] += action # 현재 스탭
    if cur_env[0] >= len(cur_data[0].index):
        cur_env[1] = data[0].iloc[len(cur_data[0].index)-1,2].copy()
    else:
        cur_env[1] = data[0].iloc[cur_env[0],2].copy() # 현재 SFR값
        
#   current state 관찰
    cur_state[0][1] = cur_state[0][0].copy() # 바로 전 z 거리
    cur_state[0][3] = cur_state[0][2].copy() # 바로 전 SFR 값
    if cur_env[0] >= len(cur_data[0].index):
        cur_state[0][0] = data[0].iloc[len(cur_data[0].index)-1,1].copy()
    else:
        cur_state[0][0] = data[0].iloc[cur_env[0],1].copy() # 현재 z 거리
    cur_state[0][2] = cur_env[1].copy() # 현재 SFR 값
    cur_state[0][4] = (cur_state[0][2]-cur_state[0][3])/(cur_state[0][1] - cur_state[0][0]) # 기울기 (reward)
    
#    reward 관찰
    reward = 0
    if cur_env[0] > cur_env[2]: # 현재 step 이 실제 best step 을 넘었으면 실패로 reward -1 주고 에피소드 종료
        terminal = True
        reward = -1
        if explor ==False:
            failed += 1  
    if cur_env[0] == cur_env[2]: # 현재 step 이 실제 best step 이면 reward 1 주고 에피소드 종료
        terminal = True
        reward = 1
        succeeded += 1
    if cur_env[0] < cur_env[2]: # 현재 step 이 실제 best step 이전이면 (기울기만큼?) reward 부여
        reward = cur_state[0][2]-cur_state[0][3]
        
#   tmp : 메모리_D 에 적재할 transition set. (cur_state, action, reward, after_state, 터미널여부) 세트로 만들어 FIFO 로 적재.
    tmp.append(reward)
    tmp.append(cur_state.copy())
    tmp.append(terminal)
    memory_D.append(tmp)
    if len(memory_D)>n_memory:
        memory_D.pop(0)
    fit_count += 1
    return cur_env, cur_state, reward, memory_D

In [15]:
def model_fit(model, model2, memory_D):
    if len(memory_D)>100:
        train_x=[]
        delt = []
        train_y=[]
#         30회 for 문돌며 memory_D에서 랜덤 추출해 학습 미니배치 구성.
        for i in range(30):
            temp = memory_D[random.randint(0,len(memory_D)-1)]
            tmp = np.zeros(5)
            tmp[temp[1]-1]=1.0

            if temp[4] == False:
                fu_reward = gamma*model2.predict_on_batch(temp[3]).max()
                delt.append(temp[2]+fu_reward)
            elif temp[4] == True:

                delt.append(temp[2])
            train_x.append(copy.deepcopy(temp[0][0]))
            train_y.append(copy.deepcopy(tmp))
        train_y = np.asarray(train_y)
        train_x = [np.asarray(train_x),np.asarray(delt), train_y]
        
#         minibatch 에 대해 1step 학습
        train_model.train_on_batch(train_x, train_y) # fit 이 아닌 train_on_batch 사용. 매우 적은 양을 학습하는 것이기 때문에 병렬처리 없애 준비시간을 줄여줌.

    return model

In [16]:
# def model_fit(model, model2, memory_D):
#     if len(memory_D)>100:
#         train_x=[]
#         delt = []
#         train_y=[]
#         for i in range(30):
#             temp = memory_D[random.randint(0,len(memory_D)-1)]
#             tmp = model.predict(temp[0])
#             if temp[4] == False:
#                 fu_reward = gamma*model2.predict(temp[3]).max()
#                 tmp[0][temp[1]-1] = temp[2]+fu_reward
#             elif temp[4] == True:

#                 tmp[0][temp[1]-1] = temp[2]
#             train_x.append(copy.deepcopy(temp[0][0]))
#             train_y.append(copy.deepcopy(tmp[0]))
#         train_x = np.asarray(train_x)
#         train_y = np.asarray(train_y)
        
#         model.fit(train_x, train_y,verbose=0)

#     return model

In [17]:
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Input, Lambda
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
def make_model():
    def custom_loss(y_true, y_pred, delta):
        out =  tf.math.square(tf.math.multiply(tf.math.subtract(y_pred, delta), y_true))
        return tf.math.reduce_mean(tf.math.reduce_sum(out, axis=1))
    delta = Input(shape=(1,),dtype= np.float32)
    y_true = Input(shape=(5,))
    x1 = Input(shape=(5,),dtype= np.float32)
    x = Dense(500,activation='relu')(x1)
    x = Dense(200,activation='relu')(x)
    x = Dense(100,activation='relu')(x)
    #     fc1 = Dense(50)(x)
    advantage = Dense(5,activation='linear')(x)
    #     fc2 = Dense(50)(x)
    value = Dense(1)(x)
    policy = Lambda(lambda x: tf.math.add(tf.math.subtract(x[0],tf.math.reduce_mean(x[0], 1, keepdims=True)),x[1]))([advantage, value])
    model = Model([x1],[policy])
    model.compile(optimizer = 'rmsprop', loss='mse')
    train_model = Model([x1, delta, y_true],[policy])
    train_model.add_loss(custom_loss(y_true, policy, delta))
    train_model.compile(optimizer = Adam(learning_rate=0.00005), loss=None)
    return model, train_model
model, train_model = make_model()
model2, train_model2 = make_model()

train_model.summary()

2023-08-08 11:59:25.890022: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-08-08 11:59:26.968942: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-08-08 11:59:27.027990: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2023-08-08 11:59:27.028617: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:86:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2023-08-08 11:59:27.028646: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 5)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 500)          3000        input_3[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 200)          100200      dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 100)          20100       dense_1[0][0]                    
____________________________________________________________________________________________

In [18]:
train_x = [np.array([[1,1,1,1,1],[1,1,1,1,1]]),np.array([[1],[1]])]
train_y = [np.array([[0,0,0,1.0,0],[0,1.0,0,0,0]])]

In [19]:
train_x = [np.array([[1,1,1,1,1],[1,1,1,1,1]]),np.array([[1],[1]])]
train_y = [np.array([[0,0,0,1,0],[0,1,0,0,0]])]
# train_model.fit(train_x,train_y,verbose=1)
model.predict([np.array([[1,1,1,1,1],[1,1,1,1,1]])])

2023-08-08 11:59:28.956083: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-08-08 11:59:28.974372: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2100000000 Hz
2023-08-08 11:59:29.073824: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-08-08 11:59:29.722734: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


array([[ 0.00288259,  0.07154439, -0.09598687,  0.07894759,  0.11638533],
       [ 0.00288259,  0.07154439, -0.09598687,  0.07894759,  0.11638533]],
      dtype=float32)

In [ ]:
from tensorflow.keras.models import load_model
# main
cur_data = random_load(train_df_list)
cur_env = init_env(cur_data) # cur_step, cur_SFR, terminal_step
# model.fit(np.asarray([np.ones(5)]),np.asarray([np.ones(5)]),verbose=0)

# model = load_model('best_model(0).h5')
# model.compile(optimizer = 'rmsprop', loss='mse', metrics=['mae'])
model2.set_weights(model.get_weights())

memory_D = []
best_acc_reward = 0
acc_step=0
acc_reward = 0
advanced_step = 0
succeeded = 0
failed = 0
action_count = 0
# with open('memory_D(1).pickle', 'rb') as f:
#     memory_D = pickle.load(f)
for j in range(1000000):
#     print(j)

#     acc_reward=0
    

    acc_step += 1
    cur_data = random_load(train_df_list)
    cur_env = init_env(cur_data)
    terminal = False
    # cur_state: [[현재 z 거리, 바로 전 z 거리, 현재 SFR, 바로 전 SFR, 기울기]]
    cur_state = np.expand_dims(np.asarray([cur_data[0].iloc[0,1], cur_data[0].iloc[0,1], cur_env[1], cur_env[1],0]),axis=0)
    reward = 0
    cur_step = 0
    episode_reward = 0
    episode_step = 0
    while terminal == False:
        action = decide(model, cur_state)
        cur_step += action
#             print(cur_step, end = ' ')
        past_state = cur_state
        cur_env, cur_state, reward, memory_D = update_env(cur_data,cur_env,action, memory_D, cur_state)

        episode_reward += reward
        episode_step += 1
        action_count += 1

        if fit_count % 6 == 0:
            model = model_fit(model, model2, memory_D)
        if fit_count % 10000 == 0:
            model2.set_weights(model.get_weights())
            print('\nmodel2 updated', j)
            print(round(acc_reward/action_count/(failed+1),2),' / ', round(best_acc_reward,2))
            print(acc_step,'번 시뮬레이션 결과')

            print('평균 개선 step 수 : ', round(advanced_step/acc_step,2))
            print('명령을 따랐는데 실패한 횟수 : ', failed)
            print('평균 Reward : ', round(acc_reward/action_count,2))
            print('exploration rate : ', exploration_rate)

            if acc_reward/action_count/(failed+1) >= best_acc_reward:
                print('best model saved')
                model.save('best_model(2).h5')
                best_acc_reward = acc_reward/action_count/(failed+1)
            acc_reward = 0
            acc_step=0
            advanced_step = 0
            succeeded = 0
            failed = 0
            action_count = 0
#         print('episode_reward : ', round(episode_reward,2))
#         print('스탭 개선 수 : ', cur_step - episode_step,'\n')
    advanced_step += cur_step - episode_step
    acc_reward += episode_reward
    exploration_rate = 0.05-0.05*fit_count/10**6
        

2023-08-08 12:01:20.586374: W tensorflow/core/kernels/data/model_dataset_op.cc:205] Optimization loop failed: Cancelled: Operation was cancelled



model2 updated 2436
0.0  /  0
2437 번 시뮬레이션 결과
평균 개선 step 수 :  4.8
명령을 따랐는데 실패한 횟수 :  179
평균 Reward :  0.33
exploration rate :  0.049500050000000004
best model saved

model2 updated 5009
0.01  /  0.0
2573 번 시뮬레이션 결과
평균 개선 step 수 :  4.84
명령을 따랐는데 실패한 횟수 :  36
평균 Reward :  0.39
exploration rate :  0.0490002
best model saved


2023-08-08 12:09:51.464598: W tensorflow/core/kernels/data/model_dataset_op.cc:205] Optimization loop failed: Cancelled: Operation was cancelled



model2 updated 7395
0.06  /  0.01
2386 번 시뮬레이션 결과
평균 개선 step 수 :  4.5
명령을 따랐는데 실패한 횟수 :  5
평균 Reward :  0.38
exploration rate :  0.04850005
best model saved


2023-08-08 12:12:58.869087: W tensorflow/core/kernels/data/model_dataset_op.cc:205] Optimization loop failed: Cancelled: Operation was cancelled


...


model2 updated 42377
0.29  /  0.34
1793 번 시뮬레이션 결과
평균 개선 step 수 :  3.33
명령을 따랐는데 실패한 횟수 :  0
평균 Reward :  0.29
exploration rate :  0.04000015


2023-08-08 13:36:03.162785: W tensorflow/core/kernels/data/model_dataset_op.cc:205] Optimization loop failed: Cancelled: Operation was cancelled
2023-08-08 13:36:03.347840: W tensorflow/core/kernels/data/model_dataset_op.cc:205] Optimization loop failed: Cancelled: Operation was cancelled
2023-08-08 13:36:21.692551: W tensorflow/core/kernels/data/model_dataset_op.cc:205] Optimization loop failed: Cancelled: Operation was cancelled
2023-08-08 13:36:21.704900: W tensorflow/core/kernels/data/model_dataset_op.cc:205] Optimization loop failed: Cancelled: Operation was cancelled
2023-08-08 13:36:21.757655: W tensorflow/core/kernels/data/model_dataset_op.cc:205] Optimization loop failed: Cancelled: Operation was cancelled
2023-08-08 13:36:21.778578: W tensorflow/core/kernels/data/model_dataset_op.cc:205] Optimization loop failed: Cancelled: Operation was cancelled
2023-08-08 13:36:21.799389: W tensorflow/core/kernels/data/model_dataset_op.cc:205] Optimization loop failed: Cancelled: Operation


model2 updated 44293
0.16  /  0.34
1916 번 시뮬레이션 결과
평균 개선 step 수 :  3.51
명령을 따랐는데 실패한 횟수 :  1
평균 Reward :  0.31
exploration rate :  0.0395002


2023-08-08 13:37:07.454639: W tensorflow/core/kernels/data/model_dataset_op.cc:205] Optimization loop failed: Cancelled: Operation was cancelled
2023-08-08 13:37:07.665693: W tensorflow/core/kernels/data/model_dataset_op.cc:205] Optimization loop failed: Cancelled: Operation was cancelled



model2 updated 46242
0.15  /  0.34
1949 번 시뮬레이션 결과
평균 개선 step 수 :  3.67
명령을 따랐는데 실패한 횟수 :  1
평균 Reward :  0.31
exploration rate :  0.03900035


In [ ]:
sdg

## 검증

In [ ]:
############## 저장된 모델 불러오는 부분########
from tensorflow.keras.models import load_model
model = load_model('best_model(2).h5')
model.compile(optimizer = 'rmsprop', loss='mse', metrics=['mae'])
###############################################

advanced_step=0
acc_reward=0
failed=0
acc_origin_step = 0
acc_episode_step=0
for k in range(len(test_df_list)):
    if k%1000==0:
        print(k,end=' ')
    cur_data = select_load(test_df_list,k)
    acc_origin_step += cur_data[1]
    cur_env = init_env(cur_data)
    terminal = False
    # cur_state: [[현재 z 거리, 바로 전 z 거리, 현재 SFR, 바로 전 SFR, 기울기]]
    cur_state = np.expand_dims(np.asarray([cur_data[0].iloc[0,1], cur_data[0].iloc[0,1], cur_env[1], cur_env[1],0]),axis=0)
    reward = 0
    cur_step = 0
    episode_reward = 0
    episode_step = 0
    
    while terminal == False:
        action = best_decide(model, cur_state)
        cur_step += action
#             print(cur_step, end = ' ')
        past_state = cur_state
        cur_env, cur_state, reward, memory_D = update_env(cur_data,cur_env,action, memory_D, cur_state)
        episode_reward += reward
        episode_step += 1
#         print('episode_reward : ', round(episode_reward,2))
#         print('스탭 개선 수 : ', cur_step - episode_step,'\n')
    advanced_step += cur_step - episode_step
    acc_episode_step += episode_step
    acc_reward += episode_reward
print('\n모든 데이터 시뮬레이션 결과 정리')
print('평균 개선 step 수 : ', advanced_step/len(test_df_list))
print('개선율 : ', 100*acc_episode_step/acc_origin_step, '%')
print('실패율 : ', 100*failed/len(test_df_list),'%')
print('명령을 따랐는데 실패한 횟수 : ', failed)
print('평균 누적 Reward : ', round(acc_reward/len(test_df_list),2),'\n')


        
    

## 아래는 데이터 전처리하였던 코드 (정리 안된 상태)

In [ ]:
count=0
idx=[]
for i in range(len(df_list)):
    if str(type(df_list[i].loc[0,'CentSFR 0.7'])) != "<class 'numpy.float64'>":
        idx.append(i)
        count+=1
        print(i)
        print(df_list[i])
#         if len(df_list[i])<10:
#             print('뷁',i)
#         if len(df_list[i])<10:
#             count+=1
#             print(i, df_list[i]['CentSFR 0.7'].argmax())
#             print(df_list[i])
print(count)

In [ ]:
df_list[idx[0]].index[10]

In [ ]:
k=0
for i in idx:
    tmp_df = df_list[i-k]
    tmp_idx=[]
    for j in range(len(tmp_df)):
        
        if tmp_df.index[j]==0:
            tmp_idx.append(j)
    for j in range(len(tmp_idx)):
        if j != len(tmp_idx)-1:
            df_list.append(tmp_df.iloc[tmp_idx[j]:tmp_idx[j+1],:])
        else:
            df_list.append(tmp_df.iloc[tmp_idx[j]:,:])
    df_list.pop(i-k)
    k+=1

In [ ]:
i

In [ ]:
for i in range(len(df_list)):
    if df_list[i].index[0]!=0:
        print(i)
        print(df_list[i])

In [ ]:
df_list.pop(48503)

In [ ]:
for i in range(len(idx)):
    if len(df_list[idx[i]].loc[0,:]) != 2:
        print(len(df_list[idx[i]].loc[0,:]))

In [ ]:
len(df_list)

In [ ]:
df_list[4316].index[0]

In [ ]:
df_list[9313].iloc[2:,2].argmax()+2

In [ ]:
df_list[9313].iloc[2:]

In [ ]:
df_list[1062]['CentSFR 0.7'].iloc[2:].argmax()

In [ ]:
df_list[1062]['CentSFR 0.7'].iloc[:]

In [ ]:
len(memory_D)